In [2]:
from gensim.models import word2vec
import MeCab
import numpy as np
import json
import hashlib
from sklearn.cross_decomposition import CCA
from googletrans import Translator
import numpy as np
from keras.layers import Dense
from keras.models import Sequential
from keras.utils import np_utils
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [3]:
translator = Translator()
mt = MeCab.Tagger('-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd/')
mt.parse('')

'EOS\n'

In [4]:
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
model_doc = Doc2Vec.load("jawiki.doc2vec.dbow300d.model")

In [5]:
def get_tags_for_doc2vec(text):
    word = []
    node = mt.parseToNode(text)
    while node:
        fields = node.feature.split(",")
        if node.surface in model_doc.wv:
            w = node.surface
            word.append(w)
        node = node.next
    return word

In [6]:
import pandas as pd
df = pd.read_excel('データ/ひろしま観光ナビ.xlsx')
df

,Unnamed: 0,観光名所,桜名所,テーマパーク,スポーツ施設,体験・見学,自然・景勝地,サイクリング,美術館・博物館,道の駅,...,観光案内所,世界遺産,ビーチ・プール,公園・キャンプ場,お土産・カフェ等,紅葉名所,スキー,クルーズ・船,フラワー,温泉・癒し・スパ
0,0,今高野山,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,1,土師ダム（八千代湖）,1,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,2,縮景園,1,0,0,0,1,0,0,0,...,0,0,0,0,1,1,0,0,1,0
3,3,錦帯橋,1,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,4,平和記念公園,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1747,1747,香遊温泉,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1748,1748,養老温泉(尾道市),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1749,1749,夢ヶ丘温泉湯めガーデン,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1750,1750,スーパー銭湯ゆ～ゆ～えびす,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [7]:
#観光名所を全てベクトル化
columns_vector = {}
for i in df:
    if i == 'Unnamed: 0' or i == '観光名所':
            continue
    else:
        columns_vector[i]= model_doc.infer_vector(get_tags_for_doc2vec(i))

In [8]:
class Label:
    ARI = 1
    NASI = 0
data = []
data_dict = {}
labels = []
for index, row in df.iterrows():
    x1 = model_doc.infer_vector(get_tags_for_doc2vec(row['観光名所']))
    for i in df:
        if i == 'Unnamed: 0' or i == '観光名所':
            continue
        else:
            x2 = columns_vector[i]
            data.extend([np.append(x1, x2)])
            data_dict[tuple(np.append(x1, x2))] = row['観光名所']+' : '+i
            if row[i] == 1:
                labels.append(Label.ARI)
            else:
                labels.append(Label.NASI)

In [9]:
data = np.array(data)
labels = np.array(labels)

In [10]:
print('data:',data)
print('labels:',labels)
print('labelの１の数:',labels.sum())

data: [[ 0.01602241  0.04063807 -0.09394898 ...  0.13490549 -0.06328392
  -0.26821196]
 [ 0.01602241  0.04063807 -0.09394898 ...  0.06719138 -0.1236745
  -0.23907642]
 [ 0.01602241  0.04063807 -0.09394898 ...  0.08607235 -0.16089243
  -0.28874913]
 ...
 [ 0.06269536  0.10972314  0.03396996 ... -0.0234566   0.01636414
  -0.3536578 ]
 [ 0.06269536  0.10972314  0.03396996 ...  0.11384635 -0.04187051
  -0.21215495]
 [ 0.06269536  0.10972314  0.03396996 ...  0.0387931  -0.19767566
  -0.24969852]]
labels: [1 0 0 ... 0 0 1]
labelの１の数: 2730


In [11]:
print('data.shape', data.shape)
print('labels.shape', labels.shape)

data.shape (33288, 600)
labels.shape (33288,)


In [12]:
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2)

In [13]:
print('トレーニングデータの全体の数:',x_train.shape[0])
print('トレーニングデータの1の数    :',y_train.sum())
print('トレーニングデータ1の割合    :',round(y_train.sum()/x_train.shape[0]*100,1),'%')
print('テストデータの全体の数         :',x_test.shape[0])
print('テストデータの１の数            :',y_test.sum())
print('テストデータの1の割合          :',round(y_test.sum()/x_test.shape[0]*100,1),'%')

トレーニングデータの全体の数: 26630
トレーニングデータの1の数    : 2185
トレーニングデータ1の割合    : 8.2 %
テストデータの全体の数         : 6658
テストデータの１の数            : 545
テストデータの1の割合          : 8.2 %


In [14]:
# y_train を one-hot 表現にする。
y_train = np_utils.to_categorical(y_train)

# モデルを作成する。
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=600)) #activation
model.add(Dense(100, activation='relu'))
model.add(Dense(2, activation='softmax'))

model.compile(
    optimizer='adam', #最適化アルゴリズムの種類.sgd,rmsprop,adam,adagrad,nadam,adadeltaなどある。
    loss='categorical_crossentropy', #損失関数名
    metrics=['accuracy']) #訓練やテストの際にモデルを評価するための評価関数のリスト． 典型的には metrics=['accuracy']を使用する.

# 学習を実行する。
model.fit(x_train, y_train, batch_size=16, epochs=30) #epochs 学習回数,batch_size 重みを調整の際に行う平均化のときに使うデータの数（dataの数/batch_size回）×epochs

# モデルを保存する。
model.save('model.h5')

# 推論する。
prob = model.predict(x_test, batch_size=len(x_test))

# 最も確率が高いのを推定クラスとする。
y_pred = np.argmax(prob, axis=1)

# 精度を検証する。
accuracy = accuracy_score(y_test, y_pred)
print('accuracy: {:.2%}'.format(accuracy))

Epoch 1/30
1665/1665 [==============================] - 1s 696us/step - loss: 0.2361 - accuracy: 0.9177
Epoch 2/30
1665/1665 [==============================] - 1s 656us/step - loss: 0.2016 - accuracy: 0.9261
Epoch 3/30
1665/1665 [==============================] - 1s 730us/step - loss: 0.1834 - accuracy: 0.92990s - loss: 0.1852 
Epoch 4/30
1665/1665 [==============================] - 1s 655us/step - loss: 0.1720 - accuracy: 0.9336
Epoch 5/30
1665/1665 [==============================] - 1s 638us/step - loss: 0.1600 - accuracy: 0.9365
Epoch 6/30
1665/1665 [==============================] - 1s 618us/step - loss: 0.1510 - accuracy: 0.9409
Epoch 7/30
1665/1665 [==============================] - 1s 634us/step - loss: 0.1424 - accuracy: 0.9441
Epoch 8/30
1665/1665 [==============================] - 1s 652us/step - loss: 0.1343 - accuracy: 0.9469
Epoch 9/30
1665/1665 [==============================] - 1s 632us/step - loss: 0.1290 - accuracy: 0.9502
Epoch 10/30
1665/1665 [=======================

In [15]:
y_test = np_utils.to_categorical(y_test)
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.295266717672348
Test accuracy: 0.9327125549316406


In [16]:
pre = model.predict(x_test)
miss_count = 0
miss_count_1 = 0
miss_0 = []
miss_1 = []
for i,v in enumerate(pre):
    pre_ans = v.argmax()
    ans = y_test[i].argmax()
    dat = x_test[i]
    if ans == pre_ans: continue
    miss_count +=1
    if ans == 1:
        miss_count_1 += 1
        miss_1.append('関連があるが関連が無いと予測    :'+data_dict[tuple(dat)])
    else:
        miss_0.append('関連が無いが関連があると予測    :'+data_dict[tuple(dat)])

In [17]:
print('予測が失敗した数:',miss_count)
print('0の予測が失敗した数:',miss_count-miss_count_1)
print('1の予測が失敗した数:',miss_count_1)

予測が失敗した数: 448
0の予測が失敗した数: 174
1の予測が失敗した数: 274


In [18]:
miss_1

['関連があるが関連が無いと予測    :鞆の浦・阿伏兎クルージング : 自然・景勝地',
 '関連があるが関連が無いと予測    :一国斉高盛金蒔絵 : お土産・カフェ等',
 '関連があるが関連が無いと予測    :織田幹雄記念館 : 美術館・博物館',
 '関連があるが関連が無いと予測    :千光寺山ロープウェイ : 公園・キャンプ場',
 '関連があるが関連が無いと予測    :独立行政法人造幣局広島支局 : 体験・見学',
 '関連があるが関連が無いと予測    :明星院 : 安全安心対策',
 '関連があるが関連が無いと予測    :宇野千代生家 : 紅葉名所',
 '関連があるが関連が無いと予測    :瀬戸の絶景\u3000大平山公園（桜名所・花見・展望） : 桜名所',
 '関連があるが関連が無いと予測    :朝比奈屋 : お土産・カフェ等',
 '関連があるが関連が無いと予測    :東広島運動公園(アクアパーク) : スポーツ施設',
 '関連があるが関連が無いと予測    :縮景園 : お土産・カフェ等',
 '関連があるが関連が無いと予測    :休暇村大久野島 : 公園・キャンプ場',
 '関連があるが関連が無いと予測    :福善寺 : 自然・景勝地',
 '関連があるが関連が無いと予測    :NHK広島放送局 : 体験・見学',
 '関連があるが関連が無いと予測    :(一社)宮島観光協会 : 観光案内所',
 '関連があるが関連が無いと予測    :【北広島町】道の駅 豊平どんぐり村 : 体験・見学',
 '関連があるが関連が無いと予測    :マリホ水族館 : 体験・見学',
 '関連があるが関連が無いと予測    :【安芸太田町】深入山グリーンシャワーオートキャンプ場 : 公園・キャンプ場',
 '関連があるが関連が無いと予測    :ラビットライン : 安全安心対策',
 '関連があるが関連が無いと予測    :出島屋内プール : ビーチ・プール',
 '関連があるが関連が無いと予測    :中央公園 : 桜名所',
 '関連があるが関連が無いと予測    :出島屋内プール : 温泉・癒し・スパ',
 '関連があるが関連が無いと予測    :嚴島神社宝物館 : 世界遺産',
 '関連があるが関連が無いと予

In [19]:
miss_0

['関連が無いが関連があると予測    :三ッ城古墳 : 紅葉名所',
 '関連が無いが関連があると予測    :みらさか竹工房\u3000はなかご : お土産・カフェ等',
 '関連が無いが関連があると予測    :㈲アリストぬまくま(そば処ぬまくま) : 体験・見学',
 '関連が無いが関連があると予測    :月見公園 : 桜名所',
 '関連が無いが関連があると予測    :祝詞山八幡神社のコバンモチ群落 : フラワー',
 '関連が無いが関連があると予測    :琴 : 安全安心対策',
 '関連が無いが関連があると予測    :【安芸太田町】滝山峡 : 紅葉名所',
 '関連が無いが関連があると予測    :県立もみのき森林公園 : 桜名所',
 '関連が無いが関連があると予測    :ツネイシしまなみビレッジ【宿泊】 : 温泉・癒し・スパ',
 '関連が無いが関連があると予測    :七塚原高原 : 安全安心対策',
 '関連が無いが関連があると予測    :みつぎグリーンランド(観光スポット) : 美術館・博物館',
 '関連が無いが関連があると予測    :今治市\u3000岩田健\u3000母と子のミュージアム : 体験・見学',
 '関連が無いが関連があると予測    :広島市グリーンスポーツセンター : ビーチ・プール',
 '関連が無いが関連があると予測    :国民宿舎みやじま杜の宿 : 道の駅',
 '関連が無いが関連があると予測    :松永グリーンパーク : 安全安心対策',
 '関連が無いが関連があると予測    :とよさか四季菜館 : 安全安心対策',
 '関連が無いが関連があると予測    :艮神社 : フラワー',
 '関連が無いが関連があると予測    :府中八幡神社 : フラワー',
 '関連が無いが関連があると予測    :吉香公園 : 紅葉名所',
 '関連が無いが関連があると予測    :二級峡 : 桜名所',
 '関連が無いが関連があると予測    :清流トレッキング体験 : 自然・景勝地',
 '関連が無いが関連があると予測    :大田庄歴史館 : 安全安心対策',
 '関連が無いが関連があると予測    :赤レンガと時計台がレトロモダンな大黒町近くにある「ソメラボ」で染め体験 : お土産・カフェ等',

In [20]:
count_1 = sum([i.argmax() for i in y_test]) #テストデータの１の数

print('全体の正解率:',1-miss_count/x_test.shape[0])
print('0の正解率    :',1-(miss_count - miss_count_1)/(x_test.shape[0]-count_1))
print('1の正解率    :',1-miss_count_1/count_1)

全体の正解率: 0.9327125262841695
0の正解率    : 0.9715360706690659
1の正解率    : 0.4972477064220183
